# BEATRICE Fine-Mapping Analysis: A Comprehensive Workflow

## Overview

This notebook demonstrates a complete fine-mapping analysis workflow using BEATRICE (Bayesian fine-mapping with trans-ethnic model selection), a state-of-the-art statistical method for identifying causal variants from genome-wide association study (GWAS) summary statistics.

## Data Requirements

This analysis requires:
1. **GWAS Summary Statistics**: Effect sizes, standard errors, and p-values
2. **Linkage Disequilibrium (LD) Matrix**: Correlation structure between variants

## Analysis Workflow

1. **Data Loading & Preprocessing**: Load and format GWAS summary statistics
2. **LD Matrix Acquisition**: Download and process LD matrices from reference panels
3. **Allele Harmonization**: Ensure consistent allele coding between datasets
4. **Quality Control**: Filter variants and validate data integrity
5. **Fine-Mapping Analysis**: Run BEATRICE algorithm
6. **Results Interpretation**: Analyze posterior probabilities and credible sets

---


## 1. Environment Setup and Dependencies

### Required Libraries
This section imports all necessary Python libraries for the analysis. Key dependencies include:
- **pandas**: Data manipulation and analysis
- **scipy**: Scientific computing, particularly for sparse matrix operations
- **numpy**: Numerical computing (imported later)
- **s3fs**: AWS S3 filesystem interface
- **gzip**: Compressed file handling


In [19]:
# Core data manipulation and analysis libraries
import pandas as pd
import numpy as np
import os

# Scientific computing libraries
from scipy.sparse import coo_matrix

# Cloud storage and data access
import s3fs
import gzip
import io

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✓ All required libraries imported successfully")
print("✓ Environment setup complete")

✓ All required libraries imported successfully
✓ Environment setup complete


## 2. Allele Harmonization Functions

### Comprehensive Allele Matching

One of the most critical steps in fine-mapping is ensuring that alleles are properly matched between the GWAS summary statistics and the LD reference panel. This function handles multiple scenarios:

**Challenge**: Summary statistics and LD reference panels may have:
- Different allele coding (A/T vs T/A)
- Different strand orientations (requiring complement matching)
- Flipped effect allele designations (requiring sign flipping)

**Solution**: The `comprehensive_allele_matching` function systematically checks:
1. **Direct Match**: A1=allele1, A2=allele2 (no adjustment needed)
2. **Flipped Match**: A1=allele2, A2=allele1 (flip beta sign)
3. **Complement Match**: A1=complement(allele1), A2=complement(allele2) (strand flip, no sign flip)
4. **Complement+Flipped**: A1=complement(allele2), A2=complement(allele1) (both strand and sign flip)

This ensures maximum variant recovery while maintaining statistical accuracy.


In [20]:
def comprehensive_allele_matching(summstats_df, ld_variants_df, complement_alleles):
    """
    Comprehensive allele matching function that handles:
    1. Direct match: A1=allele1, A2=allele2 (no sign flip)
    2. Flipped match: A1=allele2, A2=allele1 (flip beta sign)
    3. Complement match: A1=complement(allele1), A2=complement(allele2) (no sign flip)
    4. Complement+flipped match: A1=complement(allele2), A2=complement(allele1) (flip beta sign)
    
    Args:
        summstats_df: DataFrame with summary statistics containing CHR, POS, A1, A2, Beta columns
        ld_variants_df: DataFrame with LD reference containing chromosome, position, allele1, allele2 columns  
        complement_alleles: Dictionary mapping alleles to their complements
        
    Returns:
        DataFrame: Merged data with 'beta_flipped' column indicating corrected effect sizes
    """
    
    # Create a copy to avoid modifying original data
    summstats_work = summstats_df.copy()
    ld_work = ld_variants_df.copy()
    
    # Add complement alleles to both datasets
    summstats_work['A1_compl'] = summstats_work['A1'].map(complement_alleles)
    summstats_work['A2_compl'] = summstats_work['A2'].map(complement_alleles)
    
    ld_work['allele1_compl'] = ld_work['allele1'].map(complement_alleles)
    ld_work['allele2_compl'] = ld_work['allele2'].map(complement_alleles)
    
    # Initialize result list
    matched_variants = []
    
    for idx, summ_row in summstats_work.iterrows():
        chr_match = ld_work['chromosome'] == summ_row['CHR']
        pos_match = ld_work['position'] == summ_row['POS']
        chr_pos_match = chr_match & pos_match
        
        if not chr_pos_match.any():
            continue
            
        ld_candidates = ld_work[chr_pos_match]
        
        for _, ld_row in ld_candidates.iterrows():
            match_type = None
            beta_multiplier = 1  # Default: no sign flip
            
            # Scenario 1: Direct match (A1=allele1, A2=allele2)
            if (summ_row['A1'] == ld_row['allele1'] and summ_row['A2'] == ld_row['allele2']):
                match_type = 'direct'
                beta_multiplier = 1
                
            # Scenario 2: Flipped match (A1=allele2, A2=allele1)
            elif (summ_row['A1'] == ld_row['allele2'] and summ_row['A2'] == ld_row['allele1']):
                match_type = 'flipped'
                beta_multiplier = -1
                
            # Scenario 3: Complement match (A1=complement(allele1), A2=complement(allele2))
            elif (summ_row['A1'] == ld_row['allele1_compl'] and summ_row['A2'] == ld_row['allele2_compl']):
                match_type = 'complement'
                beta_multiplier = 1
                
            # Scenario 4: Complement+flipped match (A1=complement(allele2), A2=complement(allele1))
            elif (summ_row['A1'] == ld_row['allele2_compl'] and summ_row['A2'] == ld_row['allele1_compl']):
                match_type = 'complement_flipped'
                beta_multiplier = -1
                
            if match_type is not None:
                # Create merged row
                merged_row = summ_row.copy()
                
                # Add LD variant information
                for col in ld_row.index:
                    if col not in merged_row.index:
                        merged_row[col] = ld_row[col]
                        
                # Add match information
                merged_row['match_type'] = match_type
                merged_row['beta_original'] = summ_row['Beta']
                merged_row['beta_corrected'] = summ_row['Beta'] * beta_multiplier
                merged_row['sign_flipped'] = (beta_multiplier == -1)
                
                matched_variants.append(merged_row)
                break  # Take first match for each summary stat variant
    
    if len(matched_variants) == 0:
        print("Warning: No matching variants found!")
        return pd.DataFrame()
        
    result_df = pd.DataFrame(matched_variants).reset_index(drop=True)
    
    # Print matching statistics
    match_counts = result_df['match_type'].value_counts()
    print("Allele matching summary:")
    for match_type, count in match_counts.items():
        print(f"  {match_type}: {count} variants")
    print(f"Total matched variants: {len(result_df)}")
    
    return result_df


## 3. GWAS Summary Statistics Loading

### Data Source: UK Biobank High Cholesterol Study

We utilize summary statistics from the UK Biobank high cholesterol study, which provides:
- **Phenotype**: Self-reported high cholesterol (binary trait)
- **Sample Size**: ~459,000 individuals of European ancestry
- **Genome Coverage**: Genome-wide association results
- **Reference**: [Neale et al. 2020, Nature Genetics](https://www.nature.com/articles/s41588-020-00735-5)

The summary statistics include essential columns:
- `CHR`: Chromosome
- `POS`: Base pair position (GRCh37/hg19)
- `SNP`: Variant identifier
- `A1`: Effect allele
- `A2`: Reference allele  
- `Beta`: Effect size
- `se`: Standard error
- `P`: P-value
- `N`: Sample size


In [62]:
# Data source URL and reference information
data_source = "broad-alkesgroup-public-requester-pays/UKBB/disease_HI_CHOL_SELF_REP.sumstats.gz"
reference_paper = "https://www.nature.com/articles/s41588-020-00735-5"

print(f"📊 Data Source: {data_source}")
print(f"📄 Reference: {reference_paper}")
print("⚠️  Note: This dataset requires AWS requester-pays access")

📊 Data Source: broad-alkesgroup-public-requester-pays/UKBB/disease_HI_CHOL_SELF_REP.sumstats.gz
📄 Reference: https://www.nature.com/articles/s41588-020-00735-5
⚠️  Note: This dataset requires AWS requester-pays access


**Note**: Update path of summary stats

In [ ]:
# Load GWAS summary statistics
print("📂 Loading GWAS summary statistics...")
summstats_path = '/PATH/TO/UKBB_disease_HI_CHOL_SELF_REP.sumstats'

try:
    # Load the summary statistics file
    summstats = pd.read_csv(summstats_path, sep='\t')
    
    # Sort by chromosome and position for consistent ordering
    summstats = summstats.sort_values(by=['CHR', 'POS']).reset_index(drop=True)
    
    # Display basic information about the dataset
    print(f"✓ Successfully loaded {len(summstats):,} variants")
    print(f"✓ Chromosomes covered: {sorted(summstats['CHR'].unique())}")
    print(f"✓ Sample size: {summstats['N'].iloc[0]:,} individuals")
    
    # Display first few rows
    print("\n📋 First 5 variants:")
    summstats.head()
    
except FileNotFoundError:
    print(f"❌ Error: Could not find file at {summstats_path}")
    print("Please ensure the summary statistics file is downloaded and path is correct")
except Exception as e:
    print(f"❌ Error loading data: {str(e)}")

📂 Loading GWAS summary statistics...
✓ Successfully loaded 12,007,881 variants
✓ Chromosomes covered: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22)]
✓ Sample size: 459,324 individuals

📋 First 5 variants:


In [23]:
summstats.head()

,SNP,CHR,POS,A1,A2,REF,EAF,Beta,se,P,N,INFO
0,rs10399793,1,49298,T,C,T,0.376220,-0.000217,0.001172,0.85,459324,0.342797
1,rs2462492,1,54676,C,T,C,0.599409,0.001789,0.001161,0.11,459324,0.340158
2,rs3107975,1,55326,T,C,T,0.991552,-0.001885,0.006462,0.73,459324,0.324228
3,rs74447903,1,57033,T,C,T,0.998221,-0.002131,0.014378,0.84,459324,0.296256
4,1:70728_C_T,1,70728,C,T,C,0.997834,-0.009219,0.011699,0.38,459324,0.365713


## 4. Region Selection and Quality Control

### Target Region Definition

For this demonstration, we focus on a specific genomic region with significant associations:

**Region Parameters:**
- **Chromosome**: 1
- **Position Range**: 1 - 3,000,000 bp (first 3Mb of chr1)
- **Significance Threshold**: P-value ≤ 0.01

**Rationale**: 
- This region size balances computational efficiency with biological relevance
- The p-value threshold captures potentially interesting associations while maintaining manageable dataset size
- Fine-mapping is most effective in regions with clear association signals


In [31]:
# Define filtering criteria for target region
CHROMOSOME = 1
MAX_POSITION = 3_000_000  # 1Mb
P_VALUE_THRESHOLD = 0.01

print(f"🎯 Applying region filters:")
print(f"   - Chromosome: {CHROMOSOME}")
print(f"   - Position range: 1 - {MAX_POSITION:,} bp")
print(f"   - P-value threshold: ≤ {P_VALUE_THRESHOLD}")
print(f"   - Total variants before filtering: {len(summstats):,}")

🎯 Applying region filters:
   - Chromosome: 1
   - Position range: 1 - 3,000,000 bp
   - P-value threshold: ≤ 0.01
   - Total variants before filtering: 12,007,881


In [32]:
# Apply filters to extract target region
subset_summstats = summstats[
    (summstats['CHR'] == CHROMOSOME) & 
    (summstats['POS'] <= MAX_POSITION) & 
    (summstats['P'] <= P_VALUE_THRESHOLD)
].reset_index(drop=True)

# Report filtering results
print(f"✓ Filtering complete:")
print(f"   - Variants in target region: {len(subset_summstats):,}")
print(f"   - Most significant P-value: {subset_summstats['P'].min():.2e}")


✓ Filtering complete:
   - Variants in target region: 523
   - Most significant P-value: 7.70e-05


## 5. Linkage Disequilibrium (LD) Matrix Acquisition

### UK Biobank LD Reference Panel

Linkage disequilibrium matrices are essential for fine-mapping as they capture the correlation structure between variants. We use pre-computed LD matrices from the UK Biobank:

**Data Source**: [AWS Marketplace - UK Biobank LD Matrices](https://aws.amazon.com/marketplace/pp/prodview-4bhcvjnh4b4cs#overview)

**Key Features:**
- **Population**: European ancestry (matching our GWAS data)
- **Sample Size**: ~400,000 individuals
- **Format**: Sparse matrices (.npz) with variant metadata (.gz)
- **Coverage**: Genome-wide, segmented by chromosomal regions
- **Coordinate System**: GRCh37/hg19 (matching summary statistics)

**File Structure:**
- `*.npz`: Compressed sparse LD matrix (lower triangular)
- `*.gz`: Variant metadata (chromosome, position, alleles)

This ensures ancestry matching between GWAS and LD reference, critical for accurate fine-mapping.


In [63]:
# AWS S3 Configuration for LD Matrix Access
# Data source: https://aws.amazon.com/marketplace/pp/prodview-4bhcvjnh4b4cs#overview

print("🌐 Configuring AWS S3 access for LD matrices...")
print("📡 Accessing UK Biobank LD reference panel...")

🌐 Configuring AWS S3 access for LD matrices...
📡 Accessing UK Biobank LD reference panel...


In [33]:

# Initialize S3 filesystem for anonymous access
fs = s3fs.S3FileSystem(anon=True)

# Define S3 paths for target genomic region (chr1: 1-3MB)
ld_npz_s3_path = 'broad-alkesgroup-ukbb-ld/UKBB_LD/chr1_1_3000001.npz'
ld_gz_s3_path = 'broad-alkesgroup-ukbb-ld/UKBB_LD/chr1_1_3000001.gz'

print(f"📁 Target LD files:")
print(f"   - Matrix: {ld_npz_s3_path}")
print(f"   - Metadata: {ld_gz_s3_path}")

try:
    # Load LD matrix (sparse format)
    print("⬇️ Downloading LD matrix...")
    ld_data_dict = {}
    
    with fs.open(ld_npz_s3_path, 'rb') as f:
        ld_data_npz = np.load(f, allow_pickle=True)
        # Extract all data while file is still open
        for key in ld_data_npz.files:
            ld_data_dict[key] = ld_data_npz[key]
    
    print(f"✓ LD matrix loaded: {ld_data_dict['shape']} dimensions")
    
    # Load variant metadata
    print("⬇️ Downloading variant metadata...")
    with fs.open(ld_gz_s3_path, 'rb') as f:
        with gzip.open(f, 'rt') as gz_file:
            ld_variants_df = pd.read_csv(gz_file, sep='\t')
    
    print(f"✓ Variant metadata loaded: {len(ld_variants_df):,} variants")
    print(f"✓ Position range: {ld_variants_df['position'].min():,} - {ld_variants_df['position'].max():,}")
    
except Exception as e:
    print(f"❌ Error loading LD data: {str(e)}")
    print("Please check internet connection and S3 access permissions")


📁 Target LD files:
   - Matrix: broad-alkesgroup-ukbb-ld/UKBB_LD/chr1_1_3000001.npz
   - Metadata: broad-alkesgroup-ukbb-ld/UKBB_LD/chr1_1_3000001.gz
⬇️ Downloading LD matrix...
✓ LD matrix loaded: [20609 20609] dimensions
⬇️ Downloading variant metadata...
✓ Variant metadata loaded: 20,609 variants
✓ Position range: 10,177 - 2,999,890


In [34]:
# Inspect LD matrix structure
print("🔍 LD matrix components:")
for key in ld_data_dict.keys():
    data = ld_data_dict[key]
    if hasattr(data, 'shape'):
        print(f"   - {key}: {data.shape} {type(data).__name__}")
    else:
        print(f"   - {key}: {data} {type(data).__name__}")

# The sparse matrix format uses Coordinate (COO) format with:
# - 'data': correlation values
# - 'row'/'col': matrix indices 
# - 'shape': matrix dimensions
ld_data_dict.keys()

🔍 LD matrix components:
   - row: (212375732,) ndarray
   - col: (212375732,) ndarray
   - format: () ndarray
   - shape: (2,) ndarray
   - data: (212375732,) ndarray


dict_keys(['row', 'col', 'format', 'shape', 'data'])

In [35]:
# Examine LD reference variant metadata
print("📋 LD Reference Panel Variants (first 5):")
print(f"Total variants in LD panel: {len(ld_variants_df):,}")
print(f"Columns: {list(ld_variants_df.columns)}")
print("\nFirst 5 variants:")
ld_variants_df.head()

📋 LD Reference Panel Variants (first 5):
Total variants in LD panel: 20,609
Columns: ['rsid', 'chromosome', 'position', 'allele1', 'allele2']

First 5 variants:


,rsid,chromosome,position,allele1,allele2
0,rs367896724,1,10177,A,AC
1,rs201106462,1,10352,T,TA
2,rs534229142,1,10511,G,A
3,1:10616_CCGCCGTTGCAAAGGCGCGCCG_C,1,10616,CCGCCGTTGCAAAGGCGCGCCG,C
4,rs575272151,1,11008,C,G


## 6. Allele Harmonization and Variant Matching

### Critical Step: Ensuring Data Consistency

Before fine-mapping, we must harmonize alleles between the GWAS summary statistics and LD reference panel. This process is crucial because:

**Common Issues:**
- **Strand Differences**: Same variant represented on different DNA strands (A/T vs T/A)
- **Allele Flipping**: Effect allele designation differs between datasets
- **Missing Variants**: Not all GWAS variants are in the LD reference

**Our Approach:**
1. **Comprehensive Matching**: Check all possible allele combinations
2. **Sign Correction**: Flip effect size sign when alleles are swapped
3. **Quality Control**: Report matching statistics and potential issues

**Why This Matters:**
- Incorrect allele matching leads to wrong correlation patterns
- Sign errors reverse the direction of genetic effects
- Missing harmonization can exclude truly causal variants

The comprehensive allele matching function handles these complexities automatically.


In [37]:
# Define complement alleles for strand flipping
complement_alleles = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

print("🧬 Starting comprehensive allele harmonization...")
print(f"   - GWAS variants: {len(subset_summstats):,}")
print(f"   - LD reference variants: {len(ld_variants_df):,}")
print(f"   - Complement mapping: {complement_alleles}")
print("⚙️ Processing allele matching scenarios...")

🧬 Starting comprehensive allele harmonization...
   - GWAS variants: 523
   - LD reference variants: 20,609
   - Complement mapping: {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
⚙️ Processing allele matching scenarios...


In [ ]:
# Prepare LD variants DataFrame with index preservation
ld_variants_df = ld_variants_df.reset_index(drop=False)  # Keep original index for LD matrix subsetting

# Execute comprehensive allele matching
subset_summstats = comprehensive_allele_matching(
    subset_summstats, 
    ld_variants_df, 
    complement_alleles
)

# Report harmonization results
print(f"\n✅ Allele harmonization complete!")
print(f"   - Successfully matched variants: {len(subset_summstats):,}")
print(f"   - Matching rate: {len(subset_summstats) / len(subset_summstats):,.1%}")

if len(subset_summstats) > 0:
    print(f"\n📊 Harmonized dataset shape: {subset_summstats.shape}")
    print("📋 First few harmonized variants:")
    print(subset_summstats.head())
else:
    print("\n❌ Error: No variants successfully matched!")
    print("Check allele coding and coordinate systems between datasets")


Allele matching summary:
  direct: 523 variants
Total matched variants: 523

✅ Allele harmonization complete!
   - Successfully matched variants: 523
   - Matching rate: 100.0%

📊 Harmonized dataset shape: (523, 27)
📋 First few harmonized variants:
           SNP  CHR     POS A1 A2 REF       EAF      Beta        se       P  \
0   rs12562034    1  768448  G  A   G  0.893631  0.003343  0.001070  0.0016   
1  rs184266993    1  783711  G  A   G  0.994399  0.015373  0.004855  0.0020   
2   rs58233623    1  825125  C  A   C  0.998967 -0.028689  0.010606  0.0053   
3  rs190857891    1  854773  C  T   C  0.996782 -0.022571  0.007060  0.0016   
4  rs374472080    1  922812  C  T   C  0.996783  0.019318  0.006105  0.0011   

   ...  position  allele1 allele2 allele1_compl  allele2_compl match_type  \
0  ...    768448        G       A             C              T     direct   
1  ...    783711        G       A             C              T     direct   
2  ...    825125        C       A            

## 7. LD Matrix Processing and Subsetting

### Converting Sparse to Dense Matrix

The LD matrix is stored in sparse format for memory efficiency. We need to:

1. **Convert to Dense**: Transform COO sparse matrix to full dense matrix
2. **Make Symmetric**: The stored matrix is lower triangular; we create the full symmetric matrix
3. **Subset to Matched Variants**: Extract only rows/columns for successfully matched variants

**Technical Details:**
- **Sparse Format**: Only stores non-zero correlations (memory efficient)
- **Symmetry**: LD matrix is symmetric since LD(i,j) = LD(j,i)
- **Diagonal**: Perfect self-correlation (r² = 1.0)

This step ensures we have the correct correlation structure for fine-mapping.


In [40]:
# Process LD matrix from sparse to dense format
print("⚙️ Converting LD matrix from sparse to dense format...")

# Reconstruct sparse matrix from components
ld_matrix = coo_matrix(
    (ld_data_dict['data'], (ld_data_dict['row'], ld_data_dict['col'])), 
    shape=ld_data_dict['shape']
)

print(f"   - Sparse matrix shape: {ld_matrix.shape}")
print(f"   - Non-zero elements: {ld_matrix.nnz:,}")
print(f"   - Sparsity: {(1 - ld_matrix.nnz / (ld_matrix.shape[0] * ld_matrix.shape[1])):.2%}")

# Convert to dense matrix
ld_matrix_dense = ld_matrix.toarray()

# Make symmetric (original matrix is lower triangular)
print("⚙️ Creating symmetric matrix...")
ld_matrix_dense = ld_matrix_dense + ld_matrix_dense.T

print(f"✓ Dense LD matrix created: {ld_matrix_dense.shape}")
print(f"✓ Matrix is symmetric: {np.allclose(ld_matrix_dense, ld_matrix_dense.T)}")
print(f"✓ Diagonal values range: [{ld_matrix_dense.diagonal().min():.3f}, {ld_matrix_dense.diagonal().max():.3f}]")


⚙️ Converting LD matrix from sparse to dense format...
   - Sparse matrix shape: (20609, 20609)
   - Non-zero elements: 212,375,732
   - Sparsity: 50.00%
⚙️ Creating symmetric matrix...
✓ Dense LD matrix created: (20609, 20609)
✓ Matrix is symmetric: True
✓ Diagonal values range: [1.000, 1.000]


In [42]:
# Subset LD matrix to matched variants only
print("🎯 Subsetting LD matrix to matched variants...")

# Extract indices of matched variants
variant_indices = subset_summstats['index'].values
print(f"   - Extracting {len(variant_indices)} variants from LD matrix")
print(f"   - Index range: {variant_indices.min()} - {variant_indices.max()}")

# Subset both rows and columns to create final LD matrix
subset_LD = ld_matrix_dense[np.ix_(variant_indices, variant_indices)]

print(f"✓ LD matrix subsetted: {subset_LD.shape}")
print(f"✓ Matrix remains symmetric: {np.allclose(subset_LD, subset_LD.T)}")
print(f"✓ Correlation range: [{subset_LD.min():.3f}, {subset_LD.max():.3f}]")

🎯 Subsetting LD matrix to matched variants...
   - Extracting 523 variants from LD matrix
   - Index range: 1061 - 20599
✓ LD matrix subsetted: (523, 523)
✓ Matrix remains symmetric: True
✓ Correlation range: [-1.000, 1.000]


In [43]:
# Quality check: Examine LD matrix structure
print("🔍 LD Matrix Quality Check:")
print(f"   - Matrix dimensions: {subset_LD.shape}")
print(f"   - Diagonal (should be ~1.0): {subset_LD.diagonal()[:5]}")
print(f"   - Off-diagonal range: [{subset_LD[~np.eye(subset_LD.shape[0], dtype=bool)].min():.3f}, {subset_LD[~np.eye(subset_LD.shape[0], dtype=bool)].max():.3f}]")

print("\n📊 First 4x4 submatrix:")
subset_LD[:4,:4]

🔍 LD Matrix Quality Check:
   - Matrix dimensions: (523, 523)
   - Diagonal (should be ~1.0): [1. 1. 1. 1. 1.]
   - Off-diagonal range: [-1.000, 1.000]

📊 First 4x4 submatrix:


array([[ 1.0000000e+00, -2.9351717e-02, -8.3325338e-03, -1.3735899e-02],
       [-2.9351717e-02,  1.0000000e+00, -1.9033789e-03, -5.8746589e-03],
       [-8.3325338e-03, -1.9033789e-03,  1.0000000e+00, -4.2830547e-04],
       [-1.3735899e-02, -5.8746589e-03, -4.2830547e-04,  1.0000000e+00]],
      dtype=float32)

In [44]:
# Critical validation: Ensure data consistency
print("✅ Data Consistency Validation:")

# Check dimension matching
ld_variants = subset_LD.shape[0]
gwas_variants = subset_summstats.shape[0]

print(f"   - LD matrix variants: {ld_variants}")
print(f"   - GWAS variants: {gwas_variants}")

try:
    assert ld_variants == gwas_variants, f"Dimension mismatch: LD={ld_variants}, GWAS={gwas_variants}"
    print("✓ Dimensions match perfectly")
    
    # Additional validations
    assert np.all(np.isfinite(subset_LD)), "LD matrix contains non-finite values"
    assert np.all(np.isfinite(subset_summstats['beta_corrected'])), "Effect sizes contain non-finite values"
    print("✓ All values are finite")
    
    print("🎉 All validations passed - ready for fine-mapping!")
    
except AssertionError as e:
    print(f"❌ Validation failed: {e}")
    print("Please check data processing steps")

✅ Data Consistency Validation:
   - LD matrix variants: 523
   - GWAS variants: 523
✓ Dimensions match perfectly
✓ All values are finite
🎉 All validations passed - ready for fine-mapping!


## 8. BEATRICE Fine-Mapping Analysis

### Statistical Foundation

BEATRICE implements a Bayesian fine-mapping approach that:

**Core Algorithm:**
- **Bayesian Framework**: Uses prior probabilities and likelihood functions
- **Variational Inference**: Efficient approximation to full Bayesian inference  
- **Multiple Causal Variants**: Can identify multiple causal variants in a region
- **Trans-ethnic Modeling**: Handles multi-ancestry populations (when applicable)

**Input Requirements:**
- **Z-scores**: Standardized effect sizes (Beta / Standard Error)
- **LD Matrix**: Correlation structure between variants
- **Sample Size**: For proper statistical scaling

**Key Outputs:**
- **Posterior Inclusion Probabilities (PIPs)**: Probability each variant is causal
- **Credible Sets**: Sets of variants likely to contain causal variants
- **Model Selection**: Identification of the most probable causal configuration

### Data Preparation for BEATRICE

We convert our harmonized data into the format required by BEATRICE:


# Extract sample size for statistical scaling
print("📊 Extracting study parameters...")

In [45]:
# Extract sample size (should be consistent across variants)
num_samples = subset_summstats['N'].iloc[0]
sample_size_range = [subset_summstats['N'].min(), subset_summstats['N'].max()]

print(f"✓ Sample size: {num_samples:,} individuals")
print(f"✓ Sample size range: {sample_size_range[0]:,} - {sample_size_range[1]:,}")

# Verify consistent sample sizes
if sample_size_range[0] == sample_size_range[1]:
    print("✓ Sample size is consistent across all variants")
else:
    print("⚠️ Warning: Sample sizes vary across variants")
    print("   This may affect fine-mapping accuracy")

✓ Sample size: 459,324 individuals
✓ Sample size range: 459,324 - 459,324
✓ Sample size is consistent across all variants


In [46]:
# Calculate Z-scores using harmonized effect sizes
print("⚙️ Computing Z-scores for BEATRICE analysis...")

# Z-score = Effect Size / Standard Error
subset_summstats['Z'] = subset_summstats['beta_corrected'] / subset_summstats['se']

# Assign final LD matrix
LD = subset_LD

# Report allele harmonization impact
flipped_variants = subset_summstats['sign_flipped'].sum()
total_variants = len(subset_summstats)
flip_percentage = (flipped_variants / total_variants) * 100

print(f"✓ Z-scores calculated using harmonized beta values")
print(f"✓ Total variants: {total_variants:,}")
print(f"✓ Variants with flipped signs: {flipped_variants:,} ({flip_percentage:.1f}%)")
print(f"✓ Z-score range: [{subset_summstats['Z'].min():.2f}, {subset_summstats['Z'].max():.2f}]")

# Quality check
invalid_z = np.sum(~np.isfinite(subset_summstats['Z']))
if invalid_z > 0:
    print(f"⚠️ Warning: {invalid_z} variants have invalid Z-scores")
else:
    print("✓ All Z-scores are valid")

⚙️ Computing Z-scores for BEATRICE analysis...
✓ Z-scores calculated using harmonized beta values
✓ Total variants: 523
✓ Variants with flipped signs: 0 (0.0%)
✓ Z-score range: [-4.03, 3.88]
✓ All Z-scores are valid


In [47]:

# Prepare data for BEATRICE export
print("📁 Preparing BEATRICE input files...")

# Create temporary directory for input files
temp_dir = 'temp'
os.makedirs(temp_dir, exist_ok=True)
print(f"✓ Created temporary directory: {temp_dir}/")

# Prepare Z-score data (SNP ID and Z-score only)
beatrice_data = subset_summstats[['SNP', 'Z']].copy()

print(f"✓ Prepared {len(beatrice_data)} variants for analysis")
print(f"✓ Data columns: {list(beatrice_data.columns)}")


📁 Preparing BEATRICE input files...
✓ Created temporary directory: temp/
✓ Prepared 523 variants for analysis
✓ Data columns: ['SNP', 'Z']


In [48]:
# Export files for BEATRICE analysis
print("💾 Exporting input files...")

# Export Z-score file (space-separated, no header)
z_file = f"{temp_dir}/data.z"
beatrice_data.to_csv(z_file, sep=' ', index=False, header=False)
print(f"✓ Z-scores exported: {z_file}")

# Export LD matrix file (space-separated, high precision)
ld_file = f"{temp_dir}/data.ld"
np.savetxt(ld_file, LD, fmt='%.18e', delimiter=' ')
print(f"✓ LD matrix exported: {ld_file}")

# Verify file creation
z_size = os.path.getsize(z_file) / (1024**2)  # MB
ld_size = os.path.getsize(ld_file) / (1024**2)  # MB

print(f"📊 File sizes:")
print(f"   - Z-scores: {z_size:.2f} MB")
print(f"   - LD matrix: {ld_size:.2f} MB")
print("✅ Ready for BEATRICE analysis!")


💾 Exporting input files...
✓ Z-scores exported: temp/data.z
✓ LD matrix exported: temp/data.ld
📊 File sizes:
   - Z-scores: 0.01 MB
   - LD matrix: 6.66 MB
✅ Ready for BEATRICE analysis!


### Running BEATRICE Analysis

Now we execute the BEATRICE fine-mapping algorithm with our prepared data:

**Command Parameters:**
- `--z`: Z-score file (variants and effect sizes)
- `--LD`: LD matrix file (correlation structure)
- `--target`: Output directory for results
- `--N`: Sample size for proper statistical scaling
- `--plot_loss`: Generate convergence plots for diagnostics

**Expected Outputs:**
- **Posterior Inclusion Probabilities (PIPs)**: Individual variant causality probabilities
- **Credible Sets**: Groups of variants likely to contain causal variants
- **Convergence Plots**: Training loss and diagnostic visualizations
- **Summary Statistics**: Analysis metadata and quality metrics


# Execute BEATRICE fine-mapping analysis
print("🚀 Launching BEATRICE fine-mapping analysis...")
print(f"   - Input variants: {len(beatrice_data)}")
print(f"   - Sample size: {num_samples:,}")
print(f"   - Output directory: results/")
print("   - Convergence plots: Enabled")
print("\nStarting analysis... (this may take several minutes)")

In [54]:
# Run BEATRICE with optimized parameters
command = f"python ../beatrice.py --plot_loss True --z {z_file} --LD {ld_file} --target results --N {num_samples}"
print(f"Executing: {command}")
print("-" * 60)

!python ../beatrice.py --z temp/data.z --LD temp/data.ld --target results --N {num_samples}

print("-" * 60)
print("✅ BEATRICE analysis completed!")

Executing: python ../beatrice.py --plot_loss True --z temp/data.z --LD temp/data.ld --target results --N 459324
------------------------------------------------------------

 Adding a constant 0.004875433165580034 to regularize LD
100%|██████████████████████████████████████| 2001/2001 [00:04<00:00, 423.02it/s]
GENERATING CREDIBLE SETS


 Adding a constant 0.004875433165580034 to regularize LD
------------------------------------------------------------
✅ BEATRICE analysis completed!


## 9. Results Analysis and Interpretation

### Understanding BEATRICE Outputs

BEATRICE generates several key output files that provide insights into the fine-mapping results:

**Primary Output Files:**
- `pip.csv`: **Posterior Inclusion Probabilities** - The probability that each variant is causal
- `credible_set.txt`: **Credible Set Configuration** - Groups of variants that together capture causal variants with high confidence
- `credible_set.pdf`: **Credible Set Visualization** - Graphical representation of the credible sets
- `conditional_credible_variants_probability.txt`: **Conditional Analysis** - Advanced causality assessment

**Diagnostic Files:**
- `figures/`: Convergence plots and loss function visualizations
- `res`: Detailed statistical output
- `time`: Runtime performance metrics

### Key Metrics Interpretation

**Posterior Inclusion Probability (PIP):**
- **Range**: 0 to 1 (probability scale)
- **High PIP (≥0.2)**: Strong evidence for causality
- **Medium PIP (0.05-0.2)**: Moderate evidence
- **Low PIP (<0.05)**: Weak evidence


Let's examine the results systematically:


In [60]:
# Load and analyze BEATRICE results
print("📊 Loading BEATRICE results...")

results_dir = "results"
pip_file = f"{results_dir}/pip.csv"
credible_set_file = f"{results_dir}/credible_set.txt"

try:
    # Load PIP results
    pip_results = pd.read_csv(pip_file)
    print(f"✓ Loaded PIP results: {len(pip_results)} variants")
    
    # Load credible set information
    with open(credible_set_file, 'r') as f:
        credible_info = f.read().strip()
    print(f"✓ Loaded credible set configuration")
    
    # Basic statistics
    max_pip = pip_results['pip'].max()
    high_pip_count = (pip_results['pip'] >= 0.2).sum()
    medium_pip_count = ((pip_results['pip'] >= 0.05) & (pip_results['pip'] < 0.2)).sum()
    
    print(f"\n📈 Summary Statistics:")
    print(f"   - Maximum PIP: {max_pip:.3f}")
    print(f"   - High confidence variants (PIP ≥ 0.2): {high_pip_count}")
    print(f"   - Medium confidence variants (PIP 0.05-0.2): {medium_pip_count}")
    print(f"   - Total variants analyzed: {len(pip_results)}")
    
except FileNotFoundError as e:
    print(f"❌ Error loading results: {e}")
    print("Please ensure BEATRICE analysis completed successfully")


📊 Loading BEATRICE results...
✓ Loaded PIP results: 523 variants
✓ Loaded credible set configuration

📈 Summary Statistics:
   - Maximum PIP: 0.025
   - High confidence variants (PIP ≥ 0.2): 0
   - Medium confidence variants (PIP 0.05-0.2): 0
   - Total variants analyzed: 523


In [61]:
# Display top variants and credible set details
if 'pip_results' in locals():
    print("🏆 Top 10 Variants by Posterior Inclusion Probability:")
    print("=" * 70)
    
    # Sort by PIP and display top variants
    top_variants = pip_results.nlargest(10, 'pip')
    
    for i, (_, variant) in enumerate(top_variants.iterrows(), 1):
        pip_val = variant['pip']
        confidence = "HIGH" if pip_val >= 0.2 else "MEDIUM" if pip_val >= 0.05 else "LOW"
        print(f"{i:2d}. SNP: {variant['variant_names']:15s} | pip: {pip_val:.4f} | Confidence: {confidence}")
    
    print(f"\n📋 Credible Set Configuration:")
    print("=" * 50)
    print(credible_info)
    
    # PIP distribution analysis
    print(f"\n📊 PIP Distribution:")
    print("=" * 30)
    pip_values = pip_results['pip']
    print(f"   - Mean PIP: {pip_values.mean():.4f}")
    print(f"   - Median PIP: {pip_values.median():.4f}")
    print(f"   - Standard deviation: {pip_values.std():.4f}")
    print(f"   - 95th percentile: {pip_values.quantile(0.95):.4f}")
    
else:
    print("⚠️ Results not available - please run BEATRICE analysis first")


🏆 Top 10 Variants by Posterior Inclusion Probability:
 1. SNP: rs7412983       | pip: 0.0253 | Confidence: LOW
 2. SNP: rs262667        | pip: 0.0222 | Confidence: LOW
 3. SNP: rs262649        | pip: 0.0222 | Confidence: LOW
 4. SNP: rs3001800       | pip: 0.0222 | Confidence: LOW
 5. SNP: rs60435231      | pip: 0.0222 | Confidence: LOW
 6. SNP: rs55949537      | pip: 0.0222 | Confidence: LOW
 7. SNP: rs17391750      | pip: 0.0190 | Confidence: LOW
 8. SNP: rs12027883      | pip: 0.0190 | Confidence: LOW
 9. SNP: rs34408665      | pip: 0.0190 | Confidence: LOW
10. SNP: rs383968        | pip: 0.0190 | Confidence: LOW

📋 Credible Set Configuration:


📊 PIP Distribution:
   - Mean PIP: 0.0065
   - Median PIP: 0.0063
   - Standard deviation: 0.0049
   - 95th percentile: 0.0158
